In [ ]:
!pip install pandas openpyxl tkinter


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FileUpload, Button, Output

# Function to handle file upload
def handle_file_upload(change, company):
    uploaded_file = change['new']
    if uploaded_file:
        file_content = uploaded_file[next(iter(uploaded_file))]
        if company == 'A':
            global df_a
            df_a = pd.read_excel(file_content['content'])
        elif company == 'B':
            global df_b
            df_b = pd.read_excel(file_content['content'])
        output.append_stdout(f"{company} file uploaded successfully.\n")

# Create file upload widgets
file_upload_a = FileUpload(accept='.xlsx', multiple=False)
file_upload_b = FileUpload(accept='.xlsx', multiple=False)

# Bind the file upload handler
file_upload_a.observe(lambda change: handle_file_upload(change, 'A'), names='value')
file_upload_b.observe(lambda change: handle_file_upload(change, 'B'), names='value')

# Display file upload widgets
display(widgets.HTML("<b>Select the Excel file for Company A:</b>"), file_upload_a)
display(widgets.HTML("<b>Select the Excel file for Company B:</b>"), file_upload_b)

# Create button for reconciliation
reconcile_button = Button(description="Reconcile")
output = Output()

# Reconciliation logic
def reconcile_files(b):
    with output:
        output.clear_output()
        if 'df_a' in globals() and 'df_b' in globals():
            # Ensure 'Debit' and 'Credit' columns are filled with 0s where NaN
            df_a['Debit'] = df_a['Debit'].fillna(0)
            df_a['Credit'] = df_a['Credit'].fillna(0)
            df_b['Debit'] = df_b['Debit'].fillna(0)
            df_b['Credit'] = df_b['Credit'].fillna(0)

            # Filter intercompany transactions
            intercompany_a = df_a[df_a['Description'].str.contains('Company B', case=False, na=False)]
            intercompany_b = df_b[df_b['Description'].str.contains('Company A', case=False, na=False)]

            # Convert date columns to datetime
            intercompany_a['Date'] = pd.to_datetime(intercompany_a['Date'])
            intercompany_b['Date'] = pd.to_datetime(intercompany_b['Date'])

            # Merge data on Date and Description to find matches
            reconciliation = pd.merge(intercompany_a, intercompany_b, on=['Date', 'Description'], suffixes=('_A', '_B'), how='outer')

            # Identify discrepancies
            reconciliation['Debit_Diff'] = reconciliation['Debit_A'].fillna(0) - reconciliation['Debit_B'].fillna(0)
            reconciliation['Credit_Diff'] = reconciliation['Credit_A'].fillna(0) - reconciliation['Credit_B'].fillna(0)
            reconciliation['Reconciled'] = (reconciliation['Debit_Diff'] == 0) & (reconciliation['Credit_Diff'] == 0)

            # Split into discrepancies and reconciled
            discrepancies = reconciliation[~reconciliation['Reconciled']]
            reconciled = reconciliation[reconciliation['Reconciled']]

            # Save the files
            discrepancies_file_path = 'Discrepancies.xlsx'
            reconciled_file_path = 'Reconciled.xlsx'
            discrepancies.to_excel(discrepancies_file_path, index=False)
            reconciled.to_excel(reconciled_file_path, index=False)

            display(widgets.HTML(f"<b>Discrepancies file saved to:</b> {discrepancies_file_path}"))
            display(widgets.HTML(f"<b>Reconciled file saved to:</b> {reconciled_file_path}"))
        else:
            print("Please upload both files before reconciliation.")

# Bind the button to the reconciliation logic
reconcile_button.on_click(reconcile_files)

# Display the button and output
display(reconcile_button, output)


HTML(value='<b>Select the Excel file for Company A:</b>')

FileUpload(value={}, accept='.xlsx', description='Upload')

HTML(value='<b>Select the Excel file for Company B:</b>')

FileUpload(value={}, accept='.xlsx', description='Upload')

Button(description='Reconcile', style=ButtonStyle())

Output()

<ipython-input-1-6f8be51218f0>:13: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  df_a = pd.read_excel(file_content['content'])
<ipython-input-1-6f8be51218f0>:16: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  df_b = pd.read_excel(file_content['content'])
